<div align="center">
<p align="center" style="width: 100%;">
    <img src="https://raw.githubusercontent.com/vlm-run/.github/refs/heads/main/profile/assets/vlm-black.svg" alt="VLM Run Logo" width="80" style="margin-bottom: -5px; color: #2e3138; vertical-align: middle; padding-right: 5px;"><br>
</p>
<p align="center"><a href="https://docs.vlm.run"><b>Website</b></a> | <a href="https://docs.vlm.run/"><b>API Docs</b></a> | <a href="https://docs.vlm.run/blog"><b>Blog</b></a> | <a href="https://discord.gg/AMApC2UzVY"><b>Discord</b></a> | <a href="https://chat.vlm.run"><b>Chat</b></a>
</p>
</div>

# VLM Run Orion - First Aid Visual Diagnosis

This notebook demonstrates how to use [VLM Run Orion's](https://vlm.run/orion) vision capabilities to analyze an image of an injury and provide a comprehensive emergency stabilization protocol.

For more details on the API, see the [Agent API docs](https://docs.vlm.run/agents/introduction).

## Prerequisites

- Python 3.10+
- VLM Run API key (get one at [app.vlm.run](https://app.vlm.run))
- VLM Run Python Client with OpenAI extra `vlmrun[openai]`

## Setup

First, install the required packages and configure the environment.

In [8]:
# Install required packages
%pip install vlmrun[openai] --upgrade --quiet
%pip install cachetools pillow requests numpy --quiet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import os
import getpass

VLMRUN_API_KEY = os.getenv("VLMRUN_API_KEY", None)
if VLMRUN_API_KEY is None:
    VLMRUN_API_KEY = getpass.getpass("Enter your VLM Run API key: ")

## Initialize the VLM Run Client

We use the OpenAI-compatible chat completions interface through the VLM Run SDK.

In [10]:
from vlmrun.client import VLMRun

BASE_URL = os.getenv("VLMRUN_BASE_URL", "https://agent.vlm.run/v1")
client = VLMRun(api_key=VLMRUN_API_KEY, base_url=BASE_URL)
print("VLM Run client initialized successfully!")
print(f"Base URL: {BASE_URL}")

VLM Run client initialized successfully!
Base URL: https://agent.vlm.run/v1


## Response Models

We define Pydantic models for structured outputs. The response will include injury assessment, stabilization steps, safety precautions, and triage advice.

In [11]:
from pydantic import BaseModel, Field, field_validator
from typing import List, Optional, Literal


class SeverityAssessment(BaseModel):
    """Severity assessment of the injury."""
    degree: Literal["First-Degree (Superficial)", "Second-Degree (Partial-Thickness)", "Third-Degree (Full-Thickness)"] = Field(..., description="The degree/severity of the injury")
    visual_indicators: List[str] = Field(..., description="List of visual indicators supporting the severity assessment (e.g., 'redness', 'blistering', 'charring')")
    extent_percentage: Optional[int] = Field(None, description="Estimated percentage of body affected (if applicable)")
    high_risk_factors: List[str] = Field(default_factory=list, description="Critical risk factors identified (e.g., 'location on joints', 'face involvement', 'hand involvement')")

    @field_validator("high_risk_factors", mode="before")
    def _ensure_high_risk_factors(cls, v):
        if v is None:
            return []
        return v


class StabilizationStep(BaseModel):
    """A single step in the stabilization protocol."""
    step_number: int = Field(..., description="The sequential step number")
    action: str = Field(..., description="The specific action to take")
    details: str = Field(..., description="Detailed explanation of the action")
    critical: bool = Field(default=False, description="Whether this is a critical/priority action")


class SafetyPrecautions(BaseModel):
    """Safety precautions and critical do-nots."""
    do_nots: List[str] = Field(..., description="List of critical actions to avoid")
    common_mistakes: List[str] = Field(..., description="Common mistakes that can worsen the injury")
    contamination_risks: List[str] = Field(default_factory=list, description="Potential contamination or infection risks")

    @field_validator("contamination_risks", mode="before")
    def _ensure_contamination_risks(cls, v):
        if v is None:
            return []
        return v


class TriageAdvice(BaseModel):
    """Triage recommendation and criteria for professional medical intervention."""
    requires_immediate_er: bool = Field(..., description="Whether immediate ER visit is required")
    required_criteria: List[str] = Field(..., description="Specific criteria from visual evidence that necessitate professional intervention")
    transport_method: str = Field(..., description="Recommended transport method (e.g., 'Call 911 for ambulance', 'Self-transport', 'Wait for professional assessment')")
    estimated_care_level: Literal["Home Care", "Urgent Care", "Emergency Room", "Trauma Center"] = Field(..., description="Estimated level of professional care needed")


class FirstAidDiagnosisResponse(BaseModel):
    """Response containing the complete first aid diagnosis and stabilization protocol."""
    severity_assessment: SeverityAssessment = Field(..., description="Assessment of injury severity and extent")
    immediate_action_steps: List[StabilizationStep] = Field(..., description="Ordered list of stabilization steps")
    safety_precautions: SafetyPrecautions = Field(..., description="Critical do-nots and safety information")
    triage_advice: TriageAdvice = Field(..., description="Triage recommendation and professional intervention criteria")
    summary: str = Field(..., description="Overall summary of the injury assessment and recommended actions")

print("Response models defined successfully!")

Response models defined successfully!


## Helper Functions

We create helper functions to simplify making chat completion requests with structured outputs.

In [12]:
import hashlib
import json
from typing import Any, Type, TypeVar

import cachetools
from vlmrun.common.image import encode_image
from PIL import Image


T = TypeVar('T', bound=BaseModel)


def custom_key(prompt: str, image_path: str | None = None, response_model: Type[T] | None = None, model: str = "vlmrun-orion-1:auto"):
    """Custom key for caching chat_completion."""
    response_key = hashlib.sha256(json.dumps(response_model.model_json_schema(), sort_keys=True).encode()).hexdigest() if response_model else ""
    image_key = hashlib.sha256(image_path.encode()).hexdigest() if image_path else ""
    return (prompt, image_key, response_key, model)


@cachetools.cached(cache=cachetools.TTLCache(maxsize=100, ttl=3600), key=custom_key)
def chat_completion(
    prompt: str,
    image_path: str | None = None,
    response_model: Type[T] | None = None,
    model: str = "vlmrun-orion-1:auto"
) -> tuple[BaseModel | str, str]:
    """
    Make a chat completion request with structured output for first aid diagnosis.

    Args:
        prompt: The prompt describing the first aid diagnosis task
        image_path: Path to the image file containing the injury
        response_model: Pydantic model for structured output
        model: Model to use (default: vlmrun-orion-1:auto)

    Returns:
        Tuple of (parsed response model or text, session_id)
    """
    content = [{"type": "text", "text": prompt}]
    
    # Add image if provided
    if image_path:
        image = Image.open(image_path)
        image_data = encode_image(image, format="JPEG")
        content.append({"type": "image_url", "image_url": {"url": image_data}})

    kwargs = {
        "model": model,
        "messages": [{"role": "user", "content": content}]
    }

    if response_model:
        kwargs["response_format"] = {
            "type": "json_schema",
            "schema": response_model.model_json_schema()
        }

    response = client.agent.completions.create(**kwargs)
    response_text = response.choices[0].message.content
    session_id = response.session_id

    if response_model:
        result = response_model.model_validate_json(response_text)
        return result, session_id

    return response_text, session_id

print("Helper functions defined!")

Helper functions defined!


## First Aid Visual Diagnosis

Analyze an image of an injury and provide a comprehensive emergency stabilization protocol. Provide the path to an image file containing the injury.

In [13]:
# Combined prompt for first aid diagnosis
FIRST_AID_DIAGNOSIS_PROMPT = """
Task: Act as an Emergency First Aid Specialist. Analyze the attached image of an injury and provide a prioritized, step-by-step stabilization protocol.

Instructions:

Visual Assessment:
* Estimate the severity/degree of the injury (e.g., First-degree/Superficial, Second-degree/Partial-thickness, or Third-degree/Full-thickness) based on visual indicators like redness, blistering, or charring (for burns) or appropriate indicators for other injury types.
* Assess the extent of the injury and identify any high-risk factors (e.g., location on joints, face, or hands).

Immediate Action Steps:
* Provide a clear, numbered list of stabilization actions.
* Include instructions on:
  - Cooling/Temperature management: Specify the correct method (e.g., temperature of water, duration) if applicable.
  - Protection: Advise on what to use (and what not to use) for covering the wound.
  - Pain Management: Suggest non-pharmacological positioning or immediate care steps.

Safety Precautions:
* List critical "Do Nots" (e.g., "Do not pop blisters," "Do not apply ice directly," "Do not apply butter/ointments").
* Include common mistakes that can worsen the condition.

Triage Advice:
* Define the specific criteria from the visual evidence that would necessitate immediate professional medical intervention or an ER visit.
* Recommend appropriate transport method and level of professional care needed.
"""

print("First aid diagnosis prompt prepared!")
print(f"\nPrompt length: {len(FIRST_AID_DIAGNOSIS_PROMPT)} characters")

First aid diagnosis prompt prepared!

Prompt length: 1401 characters


In [ ]:
# Example: Analyze injury and provide first aid guidance
# Replace 'path/to/your/injury_image.jpg' with the actual path to your image
image_path = "burn.jpeg"

# Analyze the injury
result, session_id = chat_completion(
    prompt=FIRST_AID_DIAGNOSIS_PROMPT,
    image_path=image_path,
    response_model=FirstAidDiagnosisResponse,
    model="vlmrun-orion-1:auto"
)

print(">> RESPONSE")
print(result)
print(f"\n>> SESSION ID: {session_id}")

print("\n>> INJURY SUMMARY")
print("=" * 80)
print(result.summary)
print("=" * 80)

print("\n>> SEVERITY ASSESSMENT")
print("=" * 80)
print(f"Degree: {result.severity_assessment.degree}")
print(f"Visual Indicators: {', '.join(result.severity_assessment.visual_indicators)}")
if result.severity_assessment.extent_percentage:
    print(f"Extent: {result.severity_assessment.extent_percentage}% of body affected")
if result.severity_assessment.high_risk_factors:
    print(f"High-Risk Factors: {', '.join(result.severity_assessment.high_risk_factors)}")
print("=" * 80)

print("\n>> IMMEDIATE ACTION STEPS")
print("=" * 80)
for step in result.immediate_action_steps:
    critical_marker = " [CRITICAL]" if step.critical else ""
    print(f"\nStep {step.step_number}{critical_marker}: {step.action}")
    print(f"  Details: {step.details}")
print("=" * 80)

print("\n>> SAFETY PRECAUTIONS")
print("=" * 80)
print("Do Nots:")
for i, dont in enumerate(result.safety_precautions.do_nots, 1):
    print(f"  {i}. {dont}")
print("\nCommon Mistakes:")
for i, mistake in enumerate(result.safety_precautions.common_mistakes, 1):
    print(f"  {i}. {mistake}")
if result.safety_precautions.contamination_risks:
    print("\nContamination Risks:")
    for i, risk in enumerate(result.safety_precautions.contamination_risks, 1):
        print(f"  {i}. {risk}")
print("=" * 80)

print("\n>> TRIAGE ADVICE")
print("=" * 80)
print(f"Requires Immediate ER Visit: {'YES - CALL 911' if result.triage_advice.requires_immediate_er else 'No - Home care possible'}")
print(f"Estimated Care Level: {result.triage_advice.estimated_care_level}")
print(f"Transport Method: {result.triage_advice.transport_method}")
print("\nCriteria for Professional Intervention:")
for i, criteria in enumerate(result.triage_advice.required_criteria, 1):
    print(f"  {i}. {criteria}")
print("=" * 80)

>> RESPONSE
severity_assessment=SeverityAssessment(degree='Second-Degree (Partial-Thickness)', visual_indicators=['Fluid-filled blisters', 'Deep redness (erythema)', 'Visible swelling (edema)'], extent_percentage=None, high_risk_factors=['Location on hand', 'Involvement of finger joints', 'Functional area involvement']) immediate_action_steps=[StabilizationStep(step_number=1, action='Cool the burn with running water', details='Run cool (not cold) tap water over the affected area for at least 10 to 20 minutes to stop the burning process and reduce pain.', critical=True), StabilizationStep(step_number=2, action='Remove restrictive items', details='Gently remove rings, watches, or bracelets from the hand and wrist before swelling begins to avoid cutting off circulation.', critical=True), StabilizationStep(step_number=3, action='Cover the wound loosely', details='Use a sterile, non-stick bandage or clean plastic wrap to loosely cover the area. This protects the blisters and keeps air off t

---

## Conclusion

This notebook demonstrated how to use **VLM Run Orion** to analyze injury images and provide comprehensive emergency first aid guidance.

### Key Takeaways

1. **Structured Prompts**: Well-structured prompts with clear instructions for visual assessment, action steps, safety precautions, and triage advice produce reliable results.
2. **Session Management**: Use `session_id` to track your diagnosis requests.
3. **Image Input**: Provide clear, representative images of injuries for best results.
4. **Structured Output**: The model returns comprehensive first aid guidance in a structured format, including severity assessment, step-by-step protocols, and triage recommendations.

### Important Disclaimer

**This tool is for educational and training purposes only.** Always seek professional medical assistance for actual emergencies. In life-threatening situations, call emergency services (911 in the US) immediately. This guidance should complement, not replace, professional medical expertise.

### Next Steps

- Experiment with different types of injuries (burns, cuts, sprains, etc.)
- Use this for first aid training and certification preparation
- Explore the [VLM Run Documentation](https://docs.vlm.run) for more capabilities
- Join our [Discord community](https://discord.gg/AMApC2UzVY) for support

Stay safe and prepared! 🏥